In [1]:
import pandas as pd
import os
import numpy as np

## Normalizar nombres

### funciones especiales

In [2]:
from nombres import normalizacion_nombres

def limpiar_tildes( string ):
    if 'á' in string:
        string = string.replace('á', 'a')
    if 'é' in string:
        string = string.replace('é', 'e')
    if 'ó' in string:
        string = string.replace('ó', 'o')
    if 'í' in string:
        string = string.replace('í', 'i')
    if 'ú' in string:
        string = string.replace('ú', 'u')
    return string

def normalizar_nombres( string ):
    if string in normalizacion_nombres.keys():
        return normalizacion_nombres[string]
    return string


### ecas

In [3]:
xls_todos_eca = pd.read_excel(f"personas/data-unificada-eca.xlsx", sheet_name=['todos'])['todos']
xls_pi_eca = pd.read_excel(f"personas/data-unificada-eca.xlsx", sheet_name=['pi'])['pi']

In [4]:
personas = []
tmp_list = []
for row in xls_pi_eca[['ip', 'inst', 'email']].to_numpy().tolist():
    ip, inst, email = row
    ip = limpiar_tildes( ip.lower().strip() )
    ip = normalizar_nombres( ip )
    if ip not in personas:
        tmp_list.append( [ip, inst, email] )
        personas.append( ip )

In [5]:
for row in xls_todos_eca[['nombre', 'institucion', 'email']].to_numpy().tolist():
    ip, inst, email = row
    ip = limpiar_tildes( ip.lower().strip() )
    ip = normalizar_nombres( ip )
    if ip not in personas:
        tmp_list.append( [ip, inst, email] )
        personas.append( ip )

### procien

In [6]:
xls_asociados_procien = pd.read_excel(f"personas/detalles-asociados-procien.xlsx")
xls_proyectos_procien = pd.read_excel(f"personas/detalles-proyectos-procien.xlsx")

In [7]:
for row in xls_proyectos_procien[['nombre ip', 'institucion']].to_numpy().tolist():
    ip, inst = row
    ip = limpiar_tildes( ip.lower().strip() )
    ip = normalizar_nombres( ip )
    if ip not in personas:
        tmp_list.append( [ip, inst, 'noinf'] )
        personas.append( ip )

In [8]:
for row in xls_asociados_procien[['nombre', 'afiliacion']].to_numpy().tolist():
    ip, inst = row
    ip = limpiar_tildes( ip.lower().strip() )
    ip = normalizar_nombres( ip )
    if ip not in personas:
        tmp_list.append( [ip, inst, 'noinf'] )
        personas.append( ip )

### wos

In [9]:
xls_wos_chile = pd.read_excel(f"personas/valores-limpios-v1.xlsx")[['Authors', 'Author Full Names', 'Addresses', 'Affiliations', 'Email Addresses']]

In [ ]:
new_names = []

for row in xls_wos_chile.to_numpy().tolist():
    authors, full_name, addres, affiliations, email = row
    
    todos_nombres = [ n.strip() for n in full_name.lower().split(';') ]
    todas_afiliciaciones = [ a.strip() for a in addres.lower().split(';') if 'chile' in a]
    #print(todas_afiliciaciones)
    for nombre in todos_nombres:
        ape, nom =  nombre.split(',')[:2]
        nombre_clean = f"{nom.strip()} {ape.strip()}".lower().strip()
        nombre_clean = limpiar_tildes( nombre_clean.lower().strip() )

        for inst in todas_afiliciaciones:
            if nombre in inst:
                if nombre_clean not in personas:
                    nombre_clean = normalizar_nombres( nombre_clean )
                    tmp_list.append( [nombre_clean, inst, email] )
                    personas.append( nombre_clean )
                else:
                    new_names.append(nombre_clean)

In [11]:
full_dataframe = pd.DataFrame(np.array(tmp_list), columns=['nombre', 'institucion', 'email'])
full_dataframe.to_excel(f"personas/detalle-personas.xlsx")

## Contar

In [34]:
xls_todas = pd.read_excel(f"personas/detalle-personas.xlsx")[['nombre', 'institucion', 'email']].to_numpy().tolist()

tmp_list = []
xls_todas_personas = []#list( set( [n[0] for n in xls_todas_personas] ) )
for row in xls_todas:
    if row[0] in tmp_list:
        pass
    else:
        xls_todas_personas.append( row )
        tmp_list.append( row[0] )

In [35]:
xls_todos_eca = pd.read_excel(f"personas/data-unificada-eca.xlsx", sheet_name=['todos'])['todos'][['nombre', 'eca', 'codigo']]
xls_asociados_procien = pd.read_excel(f"personas/detalles-asociados-procien.xlsx")[['nombre']]
xls_wos_chile = pd.read_excel(f"personas/valores-limpios-v1.xlsx")[['Author Full Names']]

In [36]:
final_list = []
for (name, institucion, email) in xls_todas_personas:
    #name = name[0]
    eca_aparicion = 0
    for row in xls_todos_eca.to_numpy().tolist():
        if name == normalizar_nombres( limpiar_tildes( row[0].lower().strip() ) ):
            eca_aparicion += 1

    procien_aparicion = 0
    for row in xls_asociados_procien.to_numpy().tolist():
        if name == normalizar_nombres( limpiar_tildes( row[0].lower().strip() ) ):
            procien_aparicion += 1

    wos_aparicion = 0
    for row in xls_wos_chile.to_numpy().tolist():
        
        todos_nombres = [ n.strip() for n in row[0].lower().split(';') ]
        todos_nombres = [ f"{nom.strip()} {ape.strip()}".lower().strip() 
                for ape,nom in [n.split(',')[:2] for n in todos_nombres] ]
        todos_nombres = [ normalizar_nombres( limpiar_tildes( nom.lower().strip() ) ) for nom in todos_nombres]
        if name in todos_nombres:
            wos_aparicion +=1
    
    final_list.append( [name, institucion, email, eca_aparicion, procien_aparicion, wos_aparicion] )

In [38]:
full_dataframe = pd.DataFrame(np.array(final_list), columns=['nombre', 'institucion', 'email', 'eca', 'procien', 'wos'])
full_dataframe.to_excel(f"personas/final-personas.xlsx")